In [2]:
import torch
import torch.nn as nn
import math

In [8]:
class GroupQueryAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_key_value_heads):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads # query head个数
        self.num_key_value_heads = num_key_value_heads # key / value head个数
        assert num_heads % num_key_value_heads == 0
        self.head_dim = hidden_dim // num_heads
        assert self.head_dim % num_key_value_heads == 0
        self.q_proj = nn.Linear(hidden_dim, num_heads * self.hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, num_key_value_heads * self.head_dim)
        self.v_proj = nn.Linear(hidden_dim, num_key_value_heads * self.hidden_dim)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, X: torch.tensor, att_mask: torch.tensor = None):
        # X: (b, s, hidden_dim)
        # att_mask: (b, num_heads, s, s)

        batch_size, seq_length, _ = X.shape

        Q = self.q_proj(X)
        K = self.k_proj(X)
        V = self.v_proj(X)

        q_state = Q.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1,2)
        k_state = K.view(batch_size, seq_length, self.num_key_value_heads, self.head_dim).transpose(1,2)
        v_state = V.view(batch_size, seq_length, self.num_key_value_heads, self.head_dim).transpose(1,2)

        k_state = torch.repeat_interleave(k_state, self.num_heads // self.num_key_value_heads, dim=-1) # (batch_size, num_heads, seq_length, head_dim)
        v_state = torch.repeat_interleave(v_state, self.num_heads // self.num_key_value_heads, dim=-1) # (batch_size, num_heads, seq_length, head_dim)

        print(k_state.shape)
        print(v_state.shape)

        att_value = q_state @ k_state.transpose(-1, -2) / math.sqrt(self.head_dim)

        if att_mask is None:
            att_mask = torch.ones_like(att_value).tril()

        att_value.masked_fill_(att_mask == 0, float('-inf'))

        att_weight = torch.softmax(att_value, dim=-1) # (b, h, s, s)

        o_state = att_weight @ v_state # (b, h, s, d)

        O = self.o_proj(o_state.tranpose(1,2).contiguous().view(batch_size, seq_length, self.hidden_dim))

        return O

In [9]:
# 测试
x = torch.rand(3, 2, 128)
net = GroupQueryAttention(128, 8, 4)
net(x).shape

╭─────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────╮
│ /var/folders/nx/yz5n2z3176q0dm5gd74skwjr0000gn/T/ipykernel_75549/2243195573.py:4 in <module>                    │
│                                                                                                                 │
│ [Errno 2] No such file or directory:                                                                            │
│ '/var/folders/nx/yz5n2z3176q0dm5gd74skwjr0000gn/T/ipykernel_75549/2243195573.py'                                │
│                                                                                                                 │
│ /Users/bytedance/.miniforge3/lib/python3.10/site-packages/torch/nn/modules/module.py:1739 in _wrapped_call_impl │
│                                                                                                                 │
│   1736 │   │   if self._compiled_call_impl is not None:                                                         │
│   1737 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]                       │
│   1738 │   │   else:                                                                                            │
│ ❱ 1739 │   │   │   return self._call_impl(*args, **kwargs)                                                      │
│   1740 │                                                                                                        │
│   1741 │   # torchrec tests the code consistency with the following code                                        │
│   1742 │   # fmt: off                                                                                           │
│                                                                                                                 │
│ /Users/bytedance/.miniforge3/lib/python3.10/site-packages/torch/nn/modules/module.py:1750 in _call_impl         │
│                                                                                                                 │
│   1747 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks                  │
│   1748 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                                  │
│   1749 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                                  │
│ ❱ 1750 │   │   │   return forward_call(*args, **kwargs)                                                         │
│   1751 │   │                                                                                                    │
│   1752 │   │   result = None                                                                                    │
│   1753 │   │   called_always_called_hooks = set()                                                               │
│                                                                                                                 │
│ /var/folders/nx/yz5n2z3176q0dm5gd74skwjr0000gn/T/ipykernel_75549/3900925776.py:25 in forward                    │
│                                                                                                                 │
│ [Errno 2] No such file or directory:                                                                            │
│ '/var/folders/nx/yz5n2z3176q0dm5gd74skwjr0000gn/T/ipykernel_75549/3900925776.py'                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: shape '[3, 2, 8, 16]' is invalid for input of size 6144